<a href="https://colab.research.google.com/github/Tiabet/BaekJoon/blob/main/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-k3hweuoa/kobert-tokenizer_a3e6b0ebb6184414bca816bad8a07777
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-k3hweuoa/kobert-tokenizer_a3e6b0ebb6184414bca816bad8a07777
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=f4077b0c37896eb094f60ac5f8afb5f930c3847e3119453f77956e992ce552f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-13stw_jg/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00


In [3]:
!pip install kobert_tokenizer

In [4]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00


In [5]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7f393c109300>)

In [6]:
import pandas as pd

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력


전체 리뷰 개수 : 200000


In [7]:
total_data = total_data.drop(columns = ['ratings'])
total_data = total_data.sample(n=1000, random_state =42)

In [13]:
reviews = total_data['reviews']

In [10]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [14]:
import torch
from transformers import BertForSequenceClassification
from kobert_tokenizer import KoBERTTokenizer

# Load the model and tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1').to(device)


pred_result = []
for review in reviews:
    inputs = tokenizer.batch_encode_plus([review])

    # Convert inputs to PyTorch tensors and move to the GPU (if available)
    inputs = {key: torch.tensor(value).to(device) for key, value in inputs.items()}

    with torch.no_grad():
        out = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = out.logits

    predicted_class_id = logits.argmax().item()
    pred_result.append(predicted_class_id)

# pred_result contains the predicted class IDs
print(pred_result)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 

In [15]:
result = pd.DataFrame({'reviews' : reviews, 'Pos=0/Neg=1' : pred_result})
result.reset_index(drop=True, inplace=True)
result

,reviews,Pos=0/Neg=1
0,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,1
1,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,1
2,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,1
3,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,0
4,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,0
...,...,...
995,빠배 벌레잘 잡음 단점 대형시켯는데도 크기가 생각보다작음 대형이아니라 중형크기임,0
996,항상 여기서 사는데 s7엣지는 강화유리가 없어서 아쉽습니다,0
997,색이 사진과는 다릅니다,0
998,그냥 그래요 쏘쏘~~ 배송료 넘비싸요,0


In [16]:
result.to_csv('result_KoBERT.csv',index = False)